In [37]:
%load_ext autoreload
%autoreload 2

import spglib as spg
import ase
from ase.visualize import view

import ase.spacegroup as aspg
import numpy as np
import pandas as pd
from collections import Iterable
import cPickle as pck
import spglib as spg
import sys, os
from utils import rotation_matrix,isCellSkewed,unskewCell,get_symprec,get_relative_angle
from qe_input import makeCard,makeNamelist,SG2ibrav,SG2wyckoff,makeQEInput_sg,makeQEInput_ibrav0,makeQEInput
from custom_frame import frame2qe_format
from qe_run import run_qe_local
from raw_info import SG2BravaisLattice

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
fileNames = {}
infoPath = '../info/'
structurePath = '../structures/'
fileNames['crystals'] = structurePath + 'partial_input_crystals_sg3-230.pck'
fileNames['wyck'] = infoPath+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = infoPath+'SpaceGroup-general-info.pck'
fileNames['elements info'] = infoPath+'General-Info-Elements-fast.pck'

In [18]:
with open(fileNames['crystals'],'rb') as f:
    crystals = pck.load(f)
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

In [38]:
SG2BravaisLattice[68]

'orthorhombic C'

In [63]:
sg = 43
cc = crystals[sg][1]
dd = frame2qe_format(cc,sg)
# view(cc)
# view(dd)

In [64]:
inpt = makeQEInput(cc,sg,WyckTable,SGTable,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,
                 calculation_type='"scf"',smearing=1e-2,
                 kpt = [2,2,2],Nkpt=None,kpt_offset = [0,0,0],ppPath='"../pseudo/"',
                 PP=['Si.pbe-n-rrkjus_psl.1.0.0.UPF'])
print inpt

&CONTROL 
  calculation = "scf" 
  outdir = "./out/" 
  prefix = "qe" 
  pseudo_dir = "../pseudo/" 
  restart_mode = "from_scratch" 
  verbosity = "high" 
  wf_collect = .false. 
  nstep = 150 
  etot_conv_thr = 0.00040 
  forc_conv_thr = 0.00100 
/ 
&SYSTEM 
  ecutrho = 80.00000 
  ecutwfc = 20.00000 
  ibrav = 0 
  nat = 4 
  nbnd = 12 
  ntyp = 1 
  occupations = "smearing" 
  smearing = "cold" 
  degauss = 0.010000 
/ 
&ELECTRONS 
  conv_thr = 0.00000100 
/ 
&CELL 
  press = 0.00000 
  press_conv_thr = 0.50000 
  cell_factor = 2.00000 
/ 
ATOMIC_SPECIES   
  Si  28.0855  Si.pbe-n-rrkjus_psl.1.0.0.UPF 
ATOMIC_POSITIONS  angstrom 
  Si   2.4394518   8.01397699  1.09719521 
  Si   0.81315603  6.35925701  1.09719521 
  Si    1.62630799  11.60728549   1.86125872 
  Si   1.62629984  2.76594851  1.86125872 
K_POINTS  automatic 
  2  2  2  0  0  0 
CELL_PARAMETERS  angstrom 
   0.          7.186617    1.52812703 
   1.62630392  0.          1.52812703 
   1.62630392  7.186617    0.         

In [46]:
sg = 68
dirName = '../test_run/data_1/sg-test_'+str(sg)
cc = crystals[sg][3]
inpt = makeQEInput(cc,sg,WyckTable,SGTable,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,
                 calculation_type='"scf"',smearing=1e-2,
                pressure=0,press_conv_thr=0.5,cell_factor=2,
                etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                 kpt = [2,2,2],Nkpt=None,kpt_offset = [0,0,0],ppPath='"../pseudo/"',
                 PP=['Si.pbe-n-rrkjus_psl.1.0.0.UPF'])
exitState,error = run_qe_local(inpt,dirName,path2mpi='/usr/bin/',np=2,path2pw='/home/musil/source/qe-6.1/bin/')
print exitState
print error

0
No error


# Test symmetry detection for ibrav0

In [57]:
from qe_input import ibrav0
from tqdm import tqdm_notebook


In [58]:
dirNames = {(sg,it):'../test_run/check_ibrav0_sym/sg_{}-f_{}'.format(sg,it) 
            for sg in ibrav0 for it in range(len(crystals[sg]))}

In [60]:
exitStates,errors = {},{}
pbar = tqdm_notebook(total=len(dirNames))
for (sg,it),dirName in dirNames.iteritems():
    cc = crystals[sg][it]
    inpt = makeQEInput(cc,sg,WyckTable,SGTable,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,
                 calculation_type='"scf"',smearing=1e-2,
                pressure=0,press_conv_thr=0.5,cell_factor=2,
                etot_conv_thr=1e-2,forc_conv_thr=1e-1,nstep=150,
                scf_conv_thr=1e-1,
                 kpt = [2,2,2],Nkpt=None,kpt_offset = [0,0,0],ppPath='"../pseudo/"',
                 PP=['Si.pbe-n-rrkjus_psl.1.0.0.UPF'])
    exitStates[(sg,it)],errors[(sg,it)] = run_qe_local(inpt,dirName,path2mpi='/usr/bin/',np=2,path2pw='/home/musil/source/qe-6.1/bin/')
    pbar.update()
    if exitStates[(sg,it)]:
        print sg,it

## additional runs

In [66]:
sg = 211
dirName = '../test_run/data_1/sg-test_'+str(sg)
cc = crystals[sg][3]
inpt = makeQEInput(cc,sg,WyckTable,SGTable,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,
                 calculation_type='"scf"',smearing=1e-2,
                pressure=0,press_conv_thr=0.5,cell_factor=2,
                etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                   scf_conv_thr=1e-1,
                 kpt = [2,2,2],Nkpt=None,kpt_offset = [0,0,0],ppPath='"../pseudo/"',
                 PP=['Si.pbe-n-rrkjus_psl.1.0.0.UPF'])
exitState,error = run_qe_local(inpt,dirName,path2mpi='/usr/bin/',np=2,path2pw='/home/musil/source/qe-6.1/bin/')
print exitState
print error

0
No error


# run qe through ssh

In [67]:
from multiprocessing.dummy import Pool as ThreadedPool

source : https://gist.github.com/acdha/6064215

In [69]:
from fabric.api import 
from fabric.network import disconnect_all

# specify hostname to connect to and the remote/local paths
srcdir, remote_dirname, hostname = './qe_run.py','/home/musil','lsmx1'
try:
    s = execute(put, srcdir, remote_dirname, host=hostname)
    print(repr(s))
finally:
    disconnect_all()

[lsmx1] Executing task 'put'



Fatal error: Name lookup failed for lsmx1

Underlying exception:
    Name or service not known

Aborting.


SystemExit: 1

/home/musil/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
aa = ssh.get_host_keys

In [1]:
from fabric.api import run, env
from fabric.network import disconnect_all
env.use_ssh_config = True
env.hosts = ['fidis']


In [3]:
from fabric.api import put
put?

In [3]:
try:
    run('ls')
finally:
    disconnect_all()


[fidis] run: ls
[fidis] out: environment.yml  git  intel  miniconda	miniconda.sh  source  workspace
[fidis] out: 

Disconnecting from fidis.epfl.ch... done.


In [4]:
options = {'hostname': 'lsmx1', 'username':'musil' }

In [6]:
import paramiko,os
client = paramiko.SSHClient()
client._policy = paramiko.WarningPolicy()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh_config = paramiko.SSHConfig()
user_config_file = os.path.expanduser("~/.ssh/config")
if os.path.exists(user_config_file):
    with open(user_config_file) as f:
        ssh_config.parse(f)

cfg = {'hostname': options['hostname'], 'username': options["username"]}

user_config = ssh_config.lookup(cfg['hostname'])
for k in ('hostname', 'username', 'port'):
    if k in user_config:
        cfg[k] = user_config[k]

if 'proxycommand' in user_config:
    cfg['sock'] = paramiko.ProxyCommand(user_config['proxycommand'])

client.connect(**cfg)

In [19]:
ssh_stdin, ssh_stdout, ssh_stderr = client.exec_command('ls')
print ssh_stdout.read()

examples.desktop
fusion
Jack
miniconda2
opt
source



In [23]:
ssh_stdin, ssh_stdout, ssh_stderr = client.exec_command('echo $HOSTNAME')
print ssh_stdout.read()

AttributeError: 'NoneType' object has no attribute 'open_session'

In [22]:
client.close()